# Code used to wrangle the data and save it into smaller CSV files to save on interactive computation time:

In [1]:
import pandas as pd

## Function to retrieve the Spotify data from Github:

In [2]:
def getSpotifyData():
    """Retrieves Spotify data from Github and performs the necessary wrangling. 
    Returns:
        pandas.DataFranme: A pandas data frame with the wrangle spotify data. 
    """    
    data = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
    data = data.dropna()
    data = data.drop(['track_id', 'track_album_id', 'playlist_id'], axis=1)
    data.columns = ["Name", "Artist", "Popularity", "Album Name", "Album Release Date", "Playlist Name", 
                "Playlist Genre", "Playlist Subgenre", "Danceability", 
               "Energy", "Key", "Loudness", "Mode", "Speechiness", 
               "Acousticness", "Instrumentalness", "Liveness", "Valence", 
              "Tempo", "Duration"]
    data['Playlist Genre'] = data['Playlist Genre'].str.title()
    data['Playlist Subgenre'] = data['Playlist Subgenre'].str.title()
    mode = {1 : 'Major', 0:'Minor'}
    key = {0 : 'C', 1:'C#', 2: 'D', 3:'D#', 4: 'E', 5:'F', 6: 'F#', 7:'G', 
           8: 'G#', 9:'A', 10: 'Bb', 11:'B'}
    data.replace({"Mode":mode}, inplace = True)
    data.replace({"Key":key}, inplace = True)
    data['Album Release Date'] =  pd.to_datetime(data['Album Release Date'], format='%Y-%m-%d')
    data['Year'] = data['Album Release Date'].apply(lambda x: x.year)
    data = data.drop(["Album Release Date", "Playlist Name", "Danceability", 
               "Energy", "Key", "Loudness", "Mode", "Speechiness", 
               "Acousticness", "Instrumentalness", "Liveness", "Valence", 
              "Tempo", "Duration"], axis = 1)
    return data

In [3]:
data = getSpotifyData()

## Compute top 10 songs for every genre and year:

In [4]:
#Some songs appear multiple times, so average their popularity together. 
data_subset=data.groupby(["Name", "Artist"]).mean()
new_df = pd.DataFrame()
#Get the top 10 songs for each genre and year. 
for genre in data["Playlist Genre"].unique():
    for year in data["Year"].unique():
        new = data.loc[(data["Year"] == year) & (data["Playlist Genre"] == genre)]
        data_subset=new.groupby(["Name", "Artist", "Playlist Genre", "Year"]).mean()
        data_subset = data_subset.reset_index()
        top10=data_subset.nlargest(10,'Popularity')
        new_df = new_df.append(top10)
#Save the results into a new csv file. 
new_df.to_csv("top_songs.csv", sep = "\t", index = False)

## Compute top 10 artists for every genre and year:

In [5]:
new_df = pd.DataFrame()
#Find the top 10 for each genre and year, append to the new data frame. 
for genre in data["Playlist Genre"].unique():
    for year in data["Year"].unique():
        new = data.loc[(data["Year"] == year) & (data["Playlist Genre"] == genre)]
        data_subset=new.groupby(["Artist", "Playlist Genre", "Year"]).mean()
        data_subset = data_subset.reset_index()
        top10=data_subset.nlargest(10,'Popularity')
        new_df = new_df.append(top10)
#Save the results to a new csv file. 
new_df.to_csv("top_artists.csv", sep = "\t", index = False)

## Compute the mean popularity and number of songs released for each genre, subgenre, and year:

In [6]:
#Compute the number of records and popularity for each genre, subgenre, and year. 
counts = data.groupby(["Year", "Playlist Genre", "Playlist Subgenre"], as_index = False).size()
counts["Mean Popularity"] = (data.groupby(["Year", "Playlist Genre", "Playlist Subgenre"], as_index = False).mean("Popularity"))["Popularity"]
counts = counts.rename(columns={"size": "Number of Songs"})
#Save to a new CSV. 
counts.to_csv("by_genres.csv", sep = "\t", index = False)